<a href="https://colab.research.google.com/github/fa-key/FirstLearner/blob/main/result%20predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Input, Activation, Conv2D, MaxPooling2D, UpSampling2D, Lambda, \
Conv2DTranspose, Permute, advanced_activations, Add, LeakyReLU, Dropout, ActivityRegularization
from tensorflow.python.keras import regularizers
import matplotlib.figure as fgr
from tensorflow.python.keras import backend
from tensorflow.python.keras.callbacks import EarlyStopping
import json
import urllib

class Result():
  def fetch_data_result(self):
    updated = "updated.csv"
    self.df_beras = pd.read_csv(updated)

  @staticmethod
  def preprocess_new_data(df):
    grouped_df = df.drop(['UID'], axis=1)
    grouped_df = grouped_df.groupby(['Province']).sum()
    grouped_df.loc['Total'] = grouped_df.sum()
    grouped_df = grouped_df.T
    grouped_df.index = pd.to_datetime(grouped_df.index, infer_datetime_format=True)
    return grouped_df

  @staticmethod
  def daily_data(df):
        new_df = df.diff()
        new_df = new_df.iloc[1:, :]
        return new_df

if __name__ == "__main__":
    r = Result()
    r.fetch_data_result()



class inputBeras:
  def __init__(self):
    self.r = Result()
    self.Beras = self.r.preprocess_new_data(r.df_beras)
    self.Beras_process = self.berasProcess()
    self.X_train_list = self.splitData()

  def berasProcess(self):
    # input ke variable beras
    Beras = self.r.preprocess_new_data(r.df_beras)
    Beras_process = Beras.reset_index()

    Beras_process = Beras_process.rename(columns={'index': 'Date'})
    return Beras_process


  def splitData(self):
    # simpan data based on province ke closevalue_data dan bagi trainingdatalength 80% dri keseluruhan
    Beras_process = self.Beras_process
    training_data_lengths = []
    CloseValue_data = []
    for state in Beras_process.columns[1:]:
          Close = Beras_process.filter([state])
          CloseValue = Close.values
          TrainingDataLength = math.ceil(len(CloseValue)*.8)
          CloseValue_data.append(CloseValue)
          training_data_lengths.append(TrainingDataLength)
    CloseValue_data = np.array(CloseValue_data)
    # print(TrainingDataLength)
    # print(training_data_lengths)
    # print(CloseValue_data)

    # masukin closevalue_data ke berasdata_array
    BerasData_array = []

    scaler = MinMaxScaler(feature_range=(0, 1))

    for sample in CloseValue_data:
        # Reshape each sample assuming it's a 2D array (time_steps, features)
        reshaped_sample = sample.reshape(-1, sample.shape[-1])

        # Fit and transform each individual sample
        scaled_sample = scaler.fit_transform(reshaped_sample)

        # Reshape the scaled sample back to its original shape
        scaled_sample = scaled_sample.reshape(sample.shape)

        # Append the scaled sample to the BerasData_array
        BerasData_array.append(scaled_sample)

    BerasData_array = np.array(BerasData_array)

    # membuat x_train dan y_train tiap provinsi

    X_train_list = []  # A list to hold multiple X_train sets
    Y_train_list = []  # A list to hold corresponding Y_train sets

    for h in range(34):  # Assuming you have 34 elements in BerasData_array
        Backcandles = 15
        # len TrainData 46
        TrainData = BerasData_array[h][0:TrainingDataLength]

        X_train, Y_train = [], []  # Initialize X_train and Y_train for each iteration

        for i in range(Backcandles, len(TrainData)):
            X_train.append(TrainData[i - Backcandles:i, 0])

            Y_train.append(TrainData[i, 0])


            # if i <= Backcandles:
            #     print("X_train:", X_train, "\nY_train:", Y_train)


        X_train,Y_train = np.array(X_train), np.array(Y_train)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_train_list.append(X_train)
        Y_train_list.append(Y_train)
        return X_train_list


# pemodelan
class Regressor:
   def __init__(self, daily_df):
        self.b = inputBeras()
        self.Beras_process =  daily_df
        self.daily_df = daily_df
        self.forecast_interval = 1
        self.next_year = None

   @staticmethod
   def generate_dates(start, interval_forecast):
        index = pd.date_range(start, periods= 1  + 1, freq='Y')
        return index[1:]
   def LSTM(self, lookback=15, row='Total', num_estimators=10):
        input_shape = (self.b.X_train_list[0].shape[1], 1)
        daily_df = self.daily_df.copy()
        scaler = MinMaxScaler()
        train = daily_df
        train = np.array(train).reshape((-1, 1))
        train = scaler.fit_transform(train)
        train = train.ravel()
        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
        x_train, y_train = [], []
        for i in range(len(train) - lookback - 1 + 1):
            x = train[i:i + lookback]
            x_train.append(x)
            y_train.append(train[i + lookback:i + lookback + 1])

        train_pred = []
        for i in range(len(train) - lookback + 1):
            train_pred.append(train[i:i + lookback])

        train_pred = np.array(x_train)
        train_pred = train_pred.reshape((train_pred.shape[0], train_pred.shape[1], 1))

        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

        model = Sequential([
            LSTM(50, kernel_regularizer=regularizers.l1(0.01), input_shape=input_shape),
            Dense(self.forecast_interval)
        ])
        model.compile(optimizer="adam", loss="mean_squared_error")

        # Split the data for training and validation
        validation_split = 0.1  # You can adjust this ratio
        split_index = int(len(x_train) * (1 - validation_split))

        x_train, x_val = x_train[:split_index], x_train[split_index:]
        y_train, y_val = y_train[:split_index], y_train[split_index:]

        history = model.fit(x_train, y_train, epochs=100, verbose=1, shuffle=True,
                            validation_data=(x_val, y_val), callbacks=[callback])

        # Print the training history
        print(history.history.keys())

        x_test = np.array(y_train[-1]).reshape((1, len(y_train[-1]), 1))
        y_hat = model.predict(train_pred)

        # Flatten the predictions
        y_hat_flat = y_hat.flatten()

        y_hat = np.array(y_hat)
        y_hat = y_hat.reshape(-1, 1)
        # latest_pred = np.array(scaler.inverse_transform(model.predict(y_train[-1].reshape((1, y_train.shape[1], 1)))))
        latest_pred = scaler.inverse_transform(model.predict(y_train[-1].reshape((1, y_train.shape[1], 1))))
        ans = []
        for i in range(y_hat.shape[0]):
            temp = np.concatenate((np.array([np.nan] * i), y_hat[i],
                                   np.array([np.nan] * (y_hat.shape[0] - i + 1 - 1))), axis=0)
            ans.append(temp)

        ans.append(np.concatenate((np.array([np.nan] * (y_hat.shape[0] + 1 - 1)), latest_pred[0]),
                          axis=0))


        ans = np.array(ans)


        # print(ans.shape)
        low, high = [], []
        pred = []
        for i in range(ans.shape[1]):
            low.append(max(np.nanmin(ans[:, i]), 0))
            high.append(np.nanmax(ans[:, i]))
            pred.append(max(np.nanmean(ans[:, i]), 0))

        forecasted_days = self.generate_dates(daily_df.index[-1], 1)
        preds = pd.DataFrame(data=pred, columns=["forecast"],
                             index=daily_df.index[lookback:].union(forecasted_days))
        interval_low = pd.DataFrame(data=low, columns=["interval_low"],
                                    index=daily_df.index[lookback:].union(forecasted_days))
        interval_high = pd.DataFrame(data=high, columns=["interval_high"],
                                     index=daily_df.index[lookback:].union(forecasted_days))

        daily_df = pd.concat([daily_df, preds, interval_low, interval_high], axis=1)
        model.save("model_rice.h5")
        return daily_df


# jalanin lstm predict dan simpan ke variabel next_year dan juga ke csv tiap provinsi

   def run_predictions(self):
        # data = Data()
        # data.fetch_data()

        result = Result()
        result.fetch_data_result()

        b = inputBeras()
        Beras_process = b.Beras_process



        pred_path = "data/"

        next_year = []
        for state in Beras_process.columns[1:]:
            tempdf = Beras_process.set_index('Date')[state]
            state_reg = Regressor(tempdf)

            lstm_state_data = state_reg.LSTM(row=state, num_estimators=1)
            if (lstm_state_data['forecast'] == 0.0).any() :
              next_year.append(lstm_state_data['interval_high'].iloc[-1])
            else :
              next_year.append(lstm_state_data['forecast'].iloc[-1])
            lstm_state_data.to_csv(pred_path + "daily_{}_LSTM.csv".format(state))
        # print(next_year)
        self.next_year = next_year
        return next_year

   # masukin data prediksi tahun selanjutnya ke prediction_dict dan disimpan ke json
   def rice_predict(self, next_year):
        b = inputBeras()
        Beras_process = b.Beras_process
        next_pred = []
        latitudes = [
    4.695135, -8.4095178, -6.4058172, -3.5778471, 0.6999372, -6.211544,
    -1.4851831, -7.090911, -7.150975, -7.5360639, -0.2787808, -3.0926415,
    -1.6814878, 1.6406296, 3.07309, -2.7410513, 3.9456514,  -4.5585849, -3.2384616,
    1.5709993, -8.6529334, -8.6573819, -4.269928, -1.3361154, 0.2933469,
    -2.8441371, -3.6687994, -1.4300254, -4.14491, 0.6246932, -0.7399397,
    -3.3194374, 2.1153547, -7.8753849, 0
]
        longitudes = [
    96.7493993, 115.188916, 106.0640179, 102.3463875, 122.4467238, 106.845172,
    102.4380581, 107.668887, 110.1402594, 112.2384017, 111.4752851, 115.2837585,
    113.3823545, 116.419389, 116.04139, 106.4405872, 108.1428669, 105.4068079, 130.1452734,
    127.8087693, 117.3616476, 121.0793705, 138.0803529, 133.1747162, 101.7068294,
    119.2320784, 119.9740534, 121.4456179, 122.174605, 123.9750018, 100.8000051,
    103.914399, 99.5450974, 110.4262088, 0
]
        # Assuming next_year_predictions is a list of predictions
        # return jsonify({"predictions": next_year_predictions})
        predictions_with_province_and_location = [
    {
        "province": province,
        "prediction": prediction,
        "latitude": lat,
        "longitude": lon,
    }
    for province, prediction, lat, lon in zip(processed_data.columns[1:], next_year_predictions, latitudes, longitudes)
]
        sorted_predictions = sorted(predictions_with_province_and_location, key=lambda x: x["prediction"], reverse=True)

        ranked_predictions = [
    {
        "province": data["province"],
        "prediction": data["prediction"],
        "latitude": data["latitude"],  # Access latitude directly from data
        "longitude": data["longitude"],  # Access longitude directly from data
        "rank": rank
    }
    for rank, data in enumerate(sorted_predictions)
]
            # Appending the dictionary to the list
        next_pred.append(ranked_predictions)
        # print(next_year)
        print(next_pred)
        # Saving the list as JSON
        with open('predictions.json', 'w') as outfile:
            json.dump(next_pred, outfile, indent=4)
        return next_pred

# print(next_pred)

if __name__ == "__main__":
    r = Result()
    r.fetch_data_result()

    b = inputBeras()
    processed_data = b.Beras_process  # Access Beras_process from inputBeras

    regressor = Regressor(processed_data)
    next_year_predictions = regressor.run_predictions()

    # Pass next_year_predictions to rice_predict method
    predictions = regressor.rice_predict(next_year_predictions)

    # regressor.rice_predict()
    # predictions.save("model_rice.h5")

Epoch 1/100
2/2 [==============================] - 3s 574ms/step - loss: 0.1829 - val_loss: 0.1782
Epoch 2/100
2/2 [==============================] - 0s 38ms/step - loss: 0.1779 - val_loss: 0.1750
Epoch 3/100
2/2 [==============================] - 0s 38ms/step - loss: 0.1739 - val_loss: 0.1715
Epoch 4/100
2/2 [==============================] - 0s 38ms/step - loss: 0.1701 - val_loss: 0.1673
Epoch 5/100
2/2 [==============================] - 0s 38ms/step - loss: 0.1661 - val_loss: 0.1630
Epoch 6/100
2/2 [==============================] - 0s 40ms/step - loss: 0.1622 - val_loss: 0.1589
Epoch 7/100
2/2 [==============================] - 0s 39ms/step - loss: 0.1583 - val_loss: 0.1549
Epoch 8/100
2/2 [==============================] - 0s 38ms/step - loss: 0.1546 - val_loss: 0.1512
Epoch 9/100
2/2 [==============================] - 0s 41ms/step - loss: 0.1508 - val_loss: 0.1476
Epoch 10/100
2/2 [==============================] - 0s 38ms/step - loss: 0.1471 - val_loss: 0.1441
Epoch 11/100
2/2 [

2/2 [==============================] - 1s 7ms/step


Streaming output truncated to the last 5000 lines.
Epoch 39/100
2/2 [==============================] - 0s 47ms/step - loss: 0.0807 - val_loss: 0.0733
Epoch 40/100
2/2 [==============================] - 0s 47ms/step - loss: 0.0784 - val_loss: 0.0710
Epoch 41/100
2/2 [==============================] - 0s 45ms/step - loss: 0.0761 - val_loss: 0.0692
Epoch 42/100
2/2 [==============================] - 0s 45ms/step - loss: 0.0741 - val_loss: 0.0678
Epoch 43/100
2/2 [==============================] - 0s 46ms/step - loss: 0.0721 - val_loss: 0.0655
Epoch 44/100
2/2 [==============================] - 0s 45ms/step - loss: 0.0699 - val_loss: 0.0628
Epoch 45/100
2/2 [==============================] - 0s 50ms/step - loss: 0.0676 - val_loss: 0.0607
Epoch 46/100
2/2 [==============================] - 0s 46ms/step - loss: 0.0654 - val_loss: 0.0592
Epoch 47/100
2/2 [==============================] - 0s 47ms/step - loss: 0.0633 - val_loss: 0.0572
Epoch 48/100
2/2 [==============================] - 0s 47m

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Input, Activation, Conv2D, MaxPooling2D, UpSampling2D, Lambda, \
Conv2DTranspose, Permute, advanced_activations, Add, LeakyReLU, Dropout, ActivityRegularization
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import backend
from tensorflow.python.keras.callbacks import EarlyStopping
import json
import urllib
from flask import Flask, request, jsonify
from datetime import datetime, timedelta

class Result():

  def fetch_data_result(self):
    updated = "updated.csv"
    self.df_beras = pd.read_csv(updated)

  @staticmethod
  def preprocess_new_data(df):
    grouped_df = df.drop(['UID'], axis=1)
    grouped_df = grouped_df.groupby(['Province']).sum()
    grouped_df.loc['Total'] = grouped_df.sum()
    grouped_df = grouped_df.T
    grouped_df.index = pd.to_datetime(grouped_df.index, format="%d/%m/%Y",infer_datetime_format=True)
    return grouped_df

  @staticmethod
  def daily_data(df):
        new_df = df.diff()
        new_df = new_df.iloc[1:, :]
        return new_df

if __name__ == "__main__":
    r = Result()
    r.fetch_data_result()

class inputBeras:
  def __init__(self):
    self.r = Result()
    self.Beras = self.r.preprocess_new_data(r.df_beras)
    self.Beras_process = self.berasProcess()
    self.X_train_list = self.splitData()

  def berasProcess(self):
    # input ke variable beras
    Beras = self.r.preprocess_new_data(r.df_beras)
    Beras_process = Beras.reset_index()

    Beras_process = Beras_process.rename(columns={'index': 'Date'})
    return Beras_process


  def splitData(self):
    # simpan data based on province ke closevalue_data dan bagi trainingdatalength 80% dri keseluruhan
    Beras_process = self.Beras_process
    training_data_lengths = []
    CloseValue_data = []
    for state in Beras_process.columns[1:]:
          Close = Beras_process.filter([state])
          CloseValue = Close.values
          TrainingDataLength = math.ceil(len(CloseValue)*.8)
          CloseValue_data.append(CloseValue)
          training_data_lengths.append(TrainingDataLength)
    CloseValue_data = np.array(CloseValue_data)

    BerasData_array = []

    scaler = MinMaxScaler(feature_range=(0, 1))

    for sample in CloseValue_data:
        # Reshape each sample assuming it's a 2D array (time_steps, features)
        reshaped_sample = sample.reshape(-1, sample.shape[-1])

        # Fit and transform each individual sample
        scaled_sample = scaler.fit_transform(reshaped_sample)

        # Reshape the scaled sample back to its original shape
        scaled_sample = scaled_sample.reshape(sample.shape)

        # Append the scaled sample to the BerasData_array
        BerasData_array.append(scaled_sample)

    BerasData_array = np.array(BerasData_array)

    # membuat x_train dan y_train tiap provinsi

    X_train_list = []  # A list to hold multiple X_train sets
    Y_train_list = []  # A list to hold corresponding Y_train sets

    for h in range(34):  # Assuming you have 34 elements in BerasData_array
        Backcandles = 15
        # len TrainData 46
        TrainData = BerasData_array[h][0:TrainingDataLength]

        X_train, Y_train = [], []  # Initialize X_train and Y_train for each iteration

        for i in range(Backcandles, len(TrainData)):
            X_train.append(TrainData[i - Backcandles:i, 0])

            Y_train.append(TrainData[i, 0])


            # if i <= Backcandles:
            #     print("X_train:", X_train, "\nY_train:", Y_train)


        X_train,Y_train = np.array(X_train), np.array(Y_train)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        X_train_list.append(X_train)
        Y_train_list.append(Y_train)
        return X_train_list

class Regressor:
    def __init__(self, daily_df):
        self.b = inputBeras()
        self.Beras_process = daily_df
        self.daily_df = daily_df
        self.forecast_interval = 1
        self.next_year = None
        # self.model = self.load_model()

        # Load the pre-trained model
        self.model = self.load_model()

    def load_model(self):
        # Load your model from the saved .h5 file
        return keras.models.load_model("model.h5")

    @staticmethod
    def generate_dates(start, interval_forecast):
        index = pd.date_range(start, periods= 1  + 1, freq='Y')
        return index[1:]

    def run_predictions(self):
        next_year = []
        for state in self.Beras_process.columns[1:]:
            tempdf = self.Beras_process.set_index('Date')[state]
            state_reg = Regressor(daily_df=tempdf)
            lstm_state_data = state_reg.predicts(row=state, num_estimator=1)
            if (lstm_state_data['forecast'] == 0.0).any():
                next_year.append(lstm_state_data['interval_high'].iloc[-1])
            else:
                next_year.append(lstm_state_data['forecast'].iloc[-1])

        self.next_year = next_year
        return next_year


    def predicts(self, lookback=15, row='Total', num_estimator=10):
          train = self.daily_df
          train = np.array(train).reshape((-1, 1))
          scaler = MinMaxScaler()
          train = scaler.fit_transform(train)
          train = train.ravel()

          x_train = []
          for i in range(len(train) - lookback - 1 + 1):
              x = train[i:i + lookback]
              x_train.append(x)

          train_pred = []
          for i in range(len(train) - lookback + 1):
              train_pred.append(train[i:i + lookback])

          train_pred = np.array(x_train)
          train_pred = train_pred.reshape((train_pred.shape[0], train_pred.shape[1], 1))

          y_hat = []
          y_hat.append(scaler.inverse_transform(self.model.predict(train_pred)))

          y_hat = np.array(y_hat)
          y_hat = y_hat.reshape(-1, 1)
          latest_pred = scaler.inverse_transform(self.model.predict(train_pred[-1].reshape((1, train_pred.shape[1], 1))))
          ans = []
          for i in range(y_hat.shape[0]):
              temp = np.concatenate((np.array([np.nan] * i), y_hat[i],
                                    np.array([np.nan] * (y_hat.shape[0] - i + 1 - 1))), axis=0)
              ans.append(temp)

          ans.append(np.concatenate((np.array([np.nan] * (y_hat.shape[0] + 1 - 1)), latest_pred[0]),
                            axis=0))


          ans = np.array(ans)
          low, high = [], []
          pred = []
          for i in range(ans.shape[1]):
              low.append(max(np.nanmin(ans[:, i]), 0))
              high.append(np.nanmax(ans[:, i]))
              pred.append(max(np.nanmean(ans[:, i]), 0))

          forecasted_days = self.generate_dates(self.daily_df.index[-1], 1)
          preds = pd.DataFrame(data=pred, columns=["forecast"],
                              index=self.daily_df.index[lookback:].union(forecasted_days))
          interval_low = pd.DataFrame(data=low, columns=["interval_low"],
                                      index=self.daily_df.index[lookback:].union(forecasted_days))
          interval_high = pd.DataFrame(data=high, columns=["interval_high"],
                                      index=self.daily_df.index[lookback:].union(forecasted_days))

          daily_df = pd.concat([self.daily_df, preds, interval_low, interval_high], axis=1)
          # model.save("model_rice.h5")
          return daily_df
    # ... (rest of the class remains unchanged)

    # Create an instance of Regressor when the app starts
regressor_instance = Regressor(daily_df=None)
app = Flask(__name__)
@app.route("/prediction", methods=["GET"])
def index():
        result = Result()
        result.fetch_data_result()

        b = inputBeras()
        processed_data = b.Beras_process

        # Use the pre-trained model for predictions
        regressor_instance = Regressor(daily_df=processed_data)
        next_year_predictions = regressor_instance.run_predictions()
        next_pred = []
        latitudes = [
    4.695135, -8.4095178, -6.4058172, -3.5778471, 0.6999372, -6.211544,
    -1.4851831, -7.090911, -7.150975, -7.5360639, -0.2787808, -3.0926415,
    -1.6814878, 1.6406296, 3.07309, -2.7410513, 3.9456514,  -4.5585849, -3.2384616,
    1.5709993, -8.6529334, -8.6573819, -4.269928, -1.3361154, 0.2933469,
    -2.8441371, -3.6687994, -1.4300254, -4.14491, 0.6246932, -0.7399397,
    -3.3194374, 2.1153547, -7.8753849, 0
]
        longitudes = [
    96.7493993, 115.188916, 106.0640179, 102.3463875, 122.4467238, 106.845172,
    102.4380581, 107.668887, 110.1402594, 112.2384017, 111.4752851, 115.2837585,
    113.3823545, 116.419389, 116.04139, 106.4405872, 108.1428669, 105.4068079, 130.1452734,
    127.8087693, 117.3616476, 121.0793705, 138.0803529, 133.1747162, 101.7068294,
    119.2320784, 119.9740534, 121.4456179, 122.174605, 123.9750018, 100.8000051,
    103.914399, 99.5450974, 110.4262088, 0
]
        # Assuming next_year_predictions is a list of predictions
        # return jsonify({"predictions": next_year_predictions})
        predictions_with_province_and_location = [
    {
        "province": province,
        "prediction": prediction,
        "latitude": lat,
        "longitude": lon
    }
    for province, prediction, lat, lon in zip(processed_data.columns[1:], next_year_predictions, latitudes, longitudes)
]
            # Appending the dictionary to the list
        next_pred.append(predictions_with_province_and_location)
        # print(next_year)
        # print(next_pred)
        # Saving the list as JSON
        with open('predictions.json', 'w') as outfile:
            json.dump(next_pred, outfile, indent=4)
        return next_pred


        return jsonify(predictions_with_province)
if __name__ == "__main__":
  app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
